<a href="https://colab.research.google.com/github/ZoliN/colab/blob/main/MODNet_Inference_with_onnx.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MODNet - Inference with onnx

This notebook is the modified version of the main [colab demo](https://colab.research.google.com/drive/1GANpbKT06aEFiW-Ssx0DQnnEADcXwQG6?usp=sharing#scrollTo=JOmYOHKfgQ5Y). Refer it for more information. 


In this demo, we provide a very high performance **inference ready onnx model** for image matting. It also supports dynamic input and output shapes. <b><font color='#00FF0'>The inference time is also very less and you don't need **GPU** to run it. </font></b>

## 1. Preparation

Clone the repository and download the pretrained model.

In [ ]:
!pip install onnx onnxruntime
import os

# clone the repository
%cd /content
if not os.path.exists('MODNet'):
  !git clone https://github.com/manthan3C273/MODNet
%cd MODNet/

# dowload the onnx model for image matting
model = 'modnet.onnx'
if not os.path.exists(model):
  !gdown --id 1cgycTQlYXpTh26gB9FTnthE7AvruV8hd \
          -O pretrained/modnet.onnx

## 2. Upload image

Upload portrait images to be processed (only PNG and JPG format are supported). 

Download demo image. Photo by Charlotte May from [Pexels](https://www.pexels.com/photo/unrecognizable-asian-woman-with-rucksack-in-town-5965592/)

In [ ]:
# dowload image
image = 'image.jpg'
if not os.path.exists(image):
  !gdown --id 1fkyh03NEuSwvjFttYVwV7TjnJML04Xn6 \
          -O image.jpg

## 3. Inference

Run the following command for alpha matte prediction.

In [53]:
%cd /content/MODNet
!python demo/image_matting/Inference_with_ONNX/inference_onnx.py \
        --image-path=/content/image.jpg \
        --output-path=matte.png \
        --model-path=/content/model-small.onnx

/content/MODNet


In [ ]:
!pip install onnxsim onnxruntime
!onnxsim /content/MODNet/pretrained/modnet.onnx /content/MODNet/pretrained/modnet_672.onnx --overwrite-input-shape 1,3,672,512


In [ ]:
#!pip install onnx-tf
!python -m onnxruntime.tools.convert_onnx_models_to_ort --target_platform arm --save_optimized_onnx_model /content/model-small_16.onnx

In [ ]:
#!pip install onnx onnxconverter-common
import onnx
from onnxconverter_common import float16

model = onnx.load("/content/MODNet/pretrained/modnet.onnx")
model_fp16 = float16.convert_float_to_float16(model, keep_io_types=True)
onnx.save(model_fp16, "/content/MODNet/pretrained/modnet_16.onnx")

In [ ]:
import onnx
from onnxruntime.quantization import quantize_dynamic, QuantType

model_fp32 = '/content/MODNet/pretrained/modnet.onnx'
model_quant = '/content/MODNet/pretrained/modnetquant.onnx'
quantized_model = quantize_dynamic(model_fp32, model_quant,weight_type=QuantType.QUInt8, optimize_model=False)
#quantized_model = quantize_dynamic(model_fp32, model_quant)

## 4. Visualization

Display the results (from left to right: image, foreground, and alpha matte)

In [ ]:
import numpy as np
from PIL import Image

def combined_display(image, matte):
  # calculate display resolution
  w, h = image.width, image.height
  rw, rh = 800, int(h * 800 / (3 * w))
  
  # obtain predicted foreground
  image = np.asarray(image)
  if len(image.shape) == 2:
    image = image[:, :, None]
  if image.shape[2] == 1:
    image = np.repeat(image, 3, axis=2)
  elif image.shape[2] == 4:
    image = image[:, :, 0:3]
  matte = np.repeat(np.asarray(matte)[:, :, None], 3, axis=2) / 255
  foreground = image * matte + np.full(image.shape, 255) * (1 - matte)
  
  # combine image, foreground, and alpha into one line
  combined = np.concatenate((image, foreground, matte * 255), axis=1)
  combined = Image.fromarray(np.uint8(combined)).resize((rw, rh))
  return combined

# visualize all images

image = Image.open('image.jpg')
matte = Image.open('matte.png')
display(combined_display(image, matte))

## 5. Download image

Image with transparent background will be saved and downloaded. 

In [ ]:
image.putalpha(matte)
image.save('transparent_img.png')

from google.colab import files
files.download('transparent_img.png')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
%cd /content
#img = img[837:2700, 600:2000,:]
#img = img[1237:2700, 600:2000,:]
import cv2
import tensorflow as tf
import urllib.request
import matplotlib.pyplot as plt

url, filename = ("https://github.com/intel-isl/MiDaS/releases/download/v2/dog.jpg", "dog.jpg")
urllib.request.urlretrieve(url, filename)

url, filename = ("https://github.com/intel-isl/MiDaS/releases/download/v2_1/model_opt.tflite", "model_opt.tflite")
#urllib.request.urlretrieve(url, filename)

# input
img = cv2.imread('image.jpg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) / 255.0

img_resized = tf.image.resize(img, [256,256], method='bicubic', preserve_aspect_ratio=False)
#img_resized = tf.transpose(img_resized, [2, 0, 1])
img_input = img_resized.numpy()
mean=[0.485, 0.456, 0.406]
std=[0.229, 0.224, 0.225]
img_input = (img_input - mean) / std
reshape_img = img_input.reshape(1,256,256,3)
tensor = tf.convert_to_tensor(reshape_img, dtype=tf.float32)

# load model
#interpreter = tf.lite.Interpreter(model_path="/content/saved_model/model-small_float32.tflite")
interpreter = tf.lite.Interpreter(model_path="/content/saved_model/model_opt_float16.tflite")
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
input_shape = input_details[0]['shape']

# inference
interpreter.set_tensor(input_details[0]['index'], tensor)
interpreter.invoke()
output = interpreter.get_tensor(output_details[0]['index'])
output = output.reshape(256, 256)
             
# output file
prediction = cv2.resize(output, (img.shape[1], img.shape[0]), interpolation=cv2.INTER_CUBIC)
print(" Write image to: output.png")
depth_min = prediction.min()
depth_max = prediction.max()
img_out = (255 * (prediction - depth_min) / (depth_max - depth_min)).astype("uint8")

cv2.imwrite("output.png", img_out)
plt.imshow(img_out)

In [ ]:
%cd /content
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import cv2
from time import time

#new_model = tf.keras.models.load_model('out3')
model = tf.saved_model.load('outselfie').signatures["serving_default"] 


inpath = '/content/MODNet/image.jpg'
image = tf.io.read_file(inpath)
image = tf.image.decode_png(image, dtype=tf.uint8, channels=3)
image = image[0000:2000, 500:2000,:]
#image = image[1600:2700, 800:1900,:]
print(np.max(image))


image = (tf.image.resize(images=image, size=[256, 256]))
image = tf.cast(image, tf.float32) / 255.0
image = tf.expand_dims(image, 0)

t1 = time()
prediction = model(image)['Identity']
t2 = time()
elapsed = t2 - t1
print('Elapsed time is %f seconds.' % elapsed)

bg, prediction = tf.split(prediction, 2, 3)
#print(outp)
#prediction = tf.where(prediction > bg, 1.0, 0.0)
segres = prediction
prediction = prediction.numpy()
prediction = np.squeeze(prediction)

print(" Write image to: output.png")
depth_min = prediction.min()
depth_max = prediction.max()
img_out = (255 * (prediction - depth_min) / (depth_max - depth_min)).astype("uint8")

cv2.imwrite("depth.png", cv2.cvtColor(prediction*255.0, cv2.COLOR_BGR2RGB) )
plt.imshow(img_out)

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


In [ ]:
!wget https://github.com/isl-org/MiDaS/releases/download/v2_1/model-small.onnx

In [ ]:
!pip install onnx onnxconverter-common
import onnx
from onnxconverter_common import float16

model = onnx.load("/content/model-small.onnx")
model_fp16 = float16.convert_float_to_float16(model, keep_io_types=True)
onnx.save(model_fp16, "/content/model-small_16.onnx")

In [ ]:
!onnx2tf -i /content/model_opt.onnx  -b 1 -cotof -cotoa 1e-1 

In [ ]:
!pip install -U onnx==1.13.1 \
&& pip install -U onnxruntime==1.13.1 \
&& pip install -U onnxsim==0.4.17 \
&& pip install -U onnx2tf \
&& pip install -U h5py==3.7.0 \
&& pip install -U nvidia-pyindex \
&& pip install -U onnx-graphsurgeon \
&& pip install -U  sne4onnx \
&& pip install -U sng4onnx

#&& pip install -U simple_onnx_processing_tools \



In [ ]:
!pip install -U  sne4onnx sng4onnx
#!pip install -U onnx_tf

In [ ]:
#img = img[837:2700, 600:2000,:]
#img = img[1237:2700, 600:2000,:]
import cv2
import tensorflow as tf
import urllib.request
import matplotlib.pyplot as plt
import numpy as np


# input
img = cv2.imread('/content/image.jpg')
img = cv2.resize(img, (512,672), 0, 0, cv2.INTER_AREA)
cv2.imwrite("ds.png", img)
img = (cv2.cvtColor(img, cv2.COLOR_BGR2RGB) - 127.5) / 127.0

#img_resized = tf.image.resize(img, [672,512], method='bilinear', preserve_aspect_ratio=False)
#img_resized = tf.transpose(img_resized, [2, 0, 1])
#img_input = img_resized.numpy()

reshape_img = img.reshape(1,672,512,3)

tensor = tf.convert_to_tensor(reshape_img, dtype=tf.float32)

# load model
interpreter = tf.lite.Interpreter(model_path="/content/model.tflite")
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
input_shape = input_details[0]['shape']

# inference
interpreter.set_tensor(input_details[0]['index'], tensor)
interpreter.invoke()
output = interpreter.get_tensor(output_details[0]['index'])
output = output.reshape(672, 512)
             
# output file
prediction = output
print(" Write image to: output.png")
img_out = (255 * prediction ).astype("uint8")

cv2.imwrite("output.png", img_out)
plt.imshow(img_out)

In [61]:
from onnx import checker as ch
from onnx import helper as h
def save_new_model(opset_version, nodes, graph, out_path, verbose=True):
    if verbose:
        print("Creating new fixed graph...")
    # * create a new graph with new nodes.
    new_graph = h.make_graph(
        nodes,
        graph.name,
        graph.input,
        graph.output,
        initializer=graph.initializer,  # The initializer holds all non-constant weights.
    )
    if verbose:
        print("Creating new fixed model...")
    new_model = h.make_model(new_graph, producer_name="onnx-fix-nodes")
    new_model.opset_import[0].version = opset_version
    ch.check_model(new_model)
    if verbose:
        print(f"Saving new model as: {out_path}")
    onnx.save_model(new_model, out_path)


def fix_onnx_resize_nodes(model_path: str, out_path: str, verbose=True):
    """
    Method to fix resize nodes giving the following error in Tensorflow
    conversions:
    - "Resize coordinate_transformation_mode=pytorch_half_pixel is not supported in Tensorflow"
    """
    if verbose:
        print(f"Loading Model: {model_path}")
    # * load model.
    model = onnx.load_model(model_path)
    ch.check_model(model)
    # * get model opset version.
    opset_version = model.opset_import[0].version
    graph = model.graph

    new_nodes = []
    if verbose:
        print("Fixing Resize nodes...")
    for i, node in enumerate(graph.node):
        if node.op_type == "Resize":
            print(node)
            new_resize = onnx.helper.make_node(
                'Resize',
                inputs=node.input,
                outputs=node.output,
                name=node.name,
                coordinate_transformation_mode='half_pixel',  # Instead of pytorch_half_pixel, unsupported by Tensorflow
                mode='linear',
            )
            # Update node
            new_nodes += [new_resize]
        else:
            new_nodes += [node]

    save_new_model(opset_version=opset_version, graph=graph, nodes=new_nodes,
                   out_path=out_path, verbose=verbose)

In [ ]:
fix_onnx_resize_nodes('/content/model_opt.onnx', '/content/model_optfixed.onnx')

In [ ]:
!pip install tflite2onnx

In [ ]:
import tflite2onnx

tflite_path = '/content/model_opt.tflite'
onnx_path = '/content/model_opt.onnx'

tflite2onnx.convert(tflite_path, onnx_path)